In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import math

$x = r x (1 - x)$, где $x$ - искомое неизвестное, $r > 0$ - параметр. Корни: $x_1 = 0, x_2 = 1 - 1/r$.

In [2]:
def logistic_map(x, r):
    return r * x * (1 - x)

In [3]:
def first_x(r):
    return random.uniform ((r - 1) / (2 * r) + 1e-10, (r + 1) / (2 * r) - 1e-10)

def simple_iterations_method(r):
    eps = 1e-12
    iterations_lim = 1e7
    iterations_cur = 0
    
    x_cur  = first_x(r)
       
    x_next = logistic_map(x_cur, r)
    
    while (abs(x_next - x_cur) > eps):
        iterations_cur += 1
        x_cur  = x_next
        x_next = logistic_map(x_cur, r)
        if (iterations_cur > iterations_lim):
            return (-666, iterations_lim)
    
    return (x_next, iterations_cur)

$p(x) = r x (1 - x)$

$p'(x) = r (1 - 2x)$

**Достаточное условие сходимости**

$|p'(x)| \leq q < 1$

Выполняется на отрезке $ \frac{r - 1}{2r} < x < \frac{r + 1}{2 r}$, при условии $r > 0$

Для начала посмотрим к какому из корней $x_1$, $x_2$ сходится метод простых итераций на каждом из интервалов $(0;1)$ и $(1;3)$.


In [4]:
def simulate_for_range(left_r, right_r):
    x1_num, x2_num = 0, 0
    eps_r = 1e-10
    eps = 1e-6
    unresolved_parameters = []
    bad_accuracy = []
    
    for _ in range(10**5):
        r = random.uniform(left_r + eps_r, right_r - eps_r)
        x1 = 0.0
        x2 = 1 - (1 / r)
        (result, it) = simple_iterations_method(r)
        if (result != -666):
            if (abs(result - x1) < abs(result - x2) and abs(result - x1) < eps):
                x1_num += 1
            elif (abs(result - x2) < eps):
                x2_num += 1
            else:
                bad_accuracy.append(r)
                
        else:
            unresolved_parameters.append(r);
    return (x1_num, x2_num, bad_accuracy, unresolved_parameters)


In [ ]:
print(simulate_for_range(0, 1))
print(simulate_for_range(1, 3))


(100000, 0, [], [])


Покажем теперь численно, что:
* при $0 < r < 1$ итерационная последовательность сходится к $x_1 = 0$ монотонно;
* при $1 < r < 2$ итерационная последовательность сходится к $x_2 = 1 - \frac{1}{r}$ монотонно;
* при $2 < r < 3$ итерационная последовательность сходится к $x_2 = 1 - \frac{1}{r}$ колебательно.


In [ ]:
def simple_iterations_method_type_detection(r):
    eps = 1e-12
    iterations_lim = 1e7
    iterations_cur = 0
    side_same, side_other = 0, 0
    x_cur  = first_x(r)
    x_next = logistic_map(x_cur, r)
    
    sequence = [x_cur, x_next]
    
    while (abs(x_next - x_cur) > eps):
        iterations_cur += 1
        x_cur  = x_next
        x_next = logistic_map(x_cur, r)
        sequence.append(x_next)
        if (iterations_cur > iterations_lim):
            return (-666, -666, -666)
    
    root = x_next
    
    if (not abs(root) < 1e-8 and not abs(root - (1 - 1 / r)) < 1e-8):
        return (root, -666, -666)
    
    for i in range (0, len(sequence) - 1):
        fst = sequence[i]
        snd = sequence[i + 1]
        d = (fst - root) * (snd - root)
        if d > 0:
            side_same  += 1
        elif d < 0:
            side_other += 1
    return (root, side_same, side_other)


In [ ]:
def simulate_for_range_and_root(left_r, right_r, isZero):
    total_side_saved, total_side_changed = 0, 0
    eps_r = 1e-10
    eps = 1e-6
    errors = []

    for _ in range(10**4):
        r = random.uniform(left_r + eps_r, right_r - eps_r)
        
        if (isZero):
            root = 0
        else:
            root = 1 - 1 / r

        (res, dir_saved, dir_changed) = simple_iterations_method_type_detection(r)
        if (abs(res - root) < eps):
            total_side_saved += dir_saved
            total_side_changed += dir_changed
        else:
            errors.append((r, res))
    return (total_side_saved,  total_side_changed, errors)


In [ ]:
print(simulate_for_range_and_root(0, 1, True))
print(simulate_for_range_and_root(1, 2, False))
print(simulate_for_range_and_root(2, 3, False))

При $0 < r < 1$ сходимость к корню $x_0 = 0$. $p'(x_0) = r \Rightarrow 0 < p'(x) < r \le 1$. Таким образом, сходимость монотонная.

При $1 < r < 2$ сходимость к корню  $x_1 = 1 - \frac{1}{r}$
$p'(x_1) = 2 - r \Rightarrow 0 < p'(x) \leq 2 - r < 1$. Таким образом, сходимость монотонная.

При $2 < r < 3$ сходится в точку $x_1 = 1 - \frac{1}{r}$.
$p'(x_1) = 2 - r \Rightarrow -1 < 2 - r \leq p'(x) < 0$. Таким образом, сходимость колебательная.